# Image Classification: Malaria

In [39]:
from keras.models import Sequential, Model
from keras.layers import Reshape, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Activation, Dropout
from keras.optimizers import adadelta
from keras.callbacks import EarlyStopping

### Data Import / Preparation

Here we're dealing with quite a few images. In order to generalize the features in the images, we will zoom, scale and rotate each to add some variance in the input data. We're dealing with **quite** a bit of image data. The amount of data is pretty crazy, which means these networks should take a while to train.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
)

In [4]:
train_generator = datagen.flow_from_directory(
        'data/malaria-images',
        target_size=(110, 110),
        batch_size=32,
        class_mode='binary')

Found 27558 images belonging to 2 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    'data/malaria-images', # same directory as training data
    target_size=(110, 110),
    batch_size=32,
    class_mode='binary',
    subset='validation') # set as validation data

Found 5510 images belonging to 2 classes.


## Modeling

The next few cells will demonstrate the use of a Convolutional Neural Network to classify these images.

### Network from the groud up

This model will be generated from scratch. This structure stems from some previous research into CNNs I did a few years ago.

In [14]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(110, 110, 3)))
# add an activation layer with ReLU
model.add(Activation('relu'))
# add Dnother convolutional layer
model.add(Conv2D(32, (3,3)))
# add another activation layer now
model.add(Activation('relu'))
# perform maxPooling
model.add(MaxPooling2D(pool_size=(2,2)))
# finally add the dropout layer
# add the dropout layer now, and drop out 25%
model.add(Dropout(0.25))
# same as before just increase kernal size by 2
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
## our final layer bunch
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 110, 110, 32)      896       
_________________________________________________________________
activation_4 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 32)      9248      
_________________________________________________________________
activation_5 (Activation)    (None, 108, 108, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 54, 54, 64)        18496     
__________

In [16]:
# compile, fit and evaluate
model.compile(loss='sparse_categorical_crossentropy', optimizer=adadelta(), metrics=['accuracy'])

In [17]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, epochs=10, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 344s 2s/step - loss: 0.6986 - acc: 0.5713 - val_loss: 0.6257 - val_acc: 0.6104
Epoch 2/10
200/200 [==============================] - 346s 2s/step - loss: 0.4570 - acc: 0.8262 - val_loss: 0.2698 - val_acc: 0.9219
Epoch 3/10
200/200 [==============================] - 383s 2s/step - loss: 0.2943 - acc: 0.9134 - val_loss: 0.4069 - val_acc: 0.9325
Epoch 4/10
200/200 [==============================] - 357s 2s/step - loss: 0.2567 - acc: 0.9217 - val_loss: 0.2207 - val_acc: 0.9330
Epoch 5/10
200/200 [==============================] - 369s 2s/step - loss: 0.2154 - acc: 0.9331 - val_loss: 0.1733 - val_acc: 0.9430
Epoch 6/10
200/200 [==============================] - 345s 2s/step - loss: 0.1708 - acc: 0.9497 - val_loss: 0.1857 - val_acc: 0.9410
Epoch 7/10
200/200 [==============================] - 374s 2s/step - loss: 0.1824 - acc: 0.9469 - val_loss: 0.1771 - val_acc: 0.9389
Epoch 8/10
200/200 [==============================] - 363s 2s/step - 

### Transfer Learning: VGG19

This next model will use VGG but replace and re-train the last convolutional layer to generalize on our specific dataset. The reasoning behind this is that most of the "grunt" work has been done by the VGG model, and we're just focusing it on our domain specific content.

In [19]:
from keras import applications

# re-define the generator to comply with VGG19 specs
image_width, image_height = 256, 256
data_generator = ImageDataGenerator(
    preprocessing_function=applications.vgg19.preprocess_input,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = data_generator.flow_from_directory(
    'data/malaria-images',
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='binary'
)

validation_generator = data_generator.flow_from_directory(
    'data/malaria-images',
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='binary',
    subset="validation" # set as validation data
)


model = applications.vgg19.VGG19(weights="imagenet", include_top=False, input_shape=(image_width, image_height, 3))
print("Training Model using VGG19 with Imagenet weights...")

# add our final layers to the network
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)

predictions = Dense(train_generator.num_classes, activation="softmax")(x) # a new softmax layer to provide the downsampling needeed
# to predict which image type it is

model = Model(inputs=model.input, outputs=predictions)


# freeze all layers from being trainable except for our custom ones :)
for layer in model.layers:
    layer.trainable = False
    
# compile the model
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# allow the last layers to be trainable (these are our layers)
for layer in model.layers[17:]:
    layer.trainable = True

# re-compile
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# enable early stopping (so we don't keep trying to converge when we can't seem to anymore)
early = EarlyStopping(monitor="val_acc", min_delta=0, patience=10, verbose=1, mode="auto")


# finally fit the model and deliver the val scores
model.fit_generator(
    train_generator,
    epochs=10,
    steps_per_epoch=200,
    validation_data=validation_generator,
    validation_steps=200,
    class_weight="auto",
    use_multiprocessing=True,
    callbacks=[early]
)

Found 27558 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
Training Model using VGG19 with Imagenet weights...
Epoch 1/10
200/200 [==============================] - 3564s 18s/step - loss: 0.7511 - acc: 0.5059 - val_loss: 0.6932 - val_acc: 0.4956
Epoch 2/10
200/200 [==============================] - 3526s 18s/step - loss: 0.6934 - acc: 0.4909 - val_loss: 0.6932 - val_acc: 0.4965
Epoch 3/10
200/200 [==============================] - 3578s 18s/step - loss: 0.6930 - acc: 0.5134 - val_loss: 0.6935 - val_acc: 0.4978
Epoch 4/10
200/200 [==============================] - 3604s 18s/step - loss: 0.6934 - acc: 0.4927 - val_loss: 0.6933 - val_acc: 0.4961
Epoch 5/10
200/200 [==============================] - 3594s 18s/step - loss: 0.6932 - acc: 0.4980 - val_loss: 0.6933 - val_acc: 0.4989
Epoch 6/10
200/200 [==============================] - 3603s 18s/step - loss: 0.6933 - acc: 0.4986 - val_loss: 0.6931 - val_acc: 0.5009
Epoch 7/10
200/200 [==============================] -

### Another Model from Scratch

Not only did the transfer learning in our case have a horrid accuracy score, it also took upwards of 10 hours. That is unacceptable. I need to model to be done fairly quickly. So, for that reason, I'll keep building my own for the next 3 cells. Making incremental changes to structure, etc.

I want to try a very very small network to see if the "deepness" of the network has something to do with it. I'll use MaxPooling and Convolutional Layers, but won't use a Dropout layer in this one. (Hopefully I can see the effects of overfitting)

In [20]:
# MARK: - Instantiate the image generators

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
)

train_generator = datagen.flow_from_directory(
        'data/malaria-images',
        target_size=(110, 110),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
    'data/malaria-images', # same directory as training data
    target_size=(110, 110),
    batch_size=32,
    class_mode='binary',
    subset='validation') # set as validation data

Found 27558 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [21]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(110, 110, 3)))
# add an activation layer with ReLU
model.add(Activation('relu'))
# add Dnother convolutional layer
model.add(Conv2D(32, (3,3)))
# add another activation layer now
model.add(Activation('relu'))
# perform maxPooling
model.add(MaxPooling2D(pool_size=(2,2)))
# # finally add the dropout layer
# # add the dropout layer now, and drop out 25%
# model.add(Dropout(0.25))
# # same as before just increase kernal size by 2
# model.add(Conv2D(64, (3,3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# ## our final layer bunch
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 110, 110, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 108, 108, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 93312)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               47776256  
__________

In [23]:
# compile, fit and evaluate
model.compile(loss='sparse_categorical_crossentropy', optimizer=adadelta(), metrics=['accuracy'])

In [24]:
# Fit the model on the batches generated by datagen.flow()
model.fit_generator(train_generator, epochs=10, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 339s 2s/step - loss: 0.7706 - acc: 0.5533 - val_loss: 0.6660 - val_acc: 0.5922
Epoch 2/10
200/200 [==============================] - 307s 2s/step - loss: 0.6316 - acc: 0.6498 - val_loss: 0.6534 - val_acc: 0.6197
Epoch 3/10
200/200 [==============================] - 323s 2s/step - loss: 0.3848 - acc: 0.8575 - val_loss: 0.2611 - val_acc: 0.9176
Epoch 4/10
200/200 [==============================] - 296s 1s/step - loss: 0.2535 - acc: 0.9172 - val_loss: 0.3236 - val_acc: 0.9264
Epoch 5/10
200/200 [==============================] - 407s 2s/step - loss: 0.2324 - acc: 0.9214 - val_loss: 0.2111 - val_acc: 0.9264
Epoch 6/10
200/200 [==============================] - 269s 1s/step - loss: 0.2165 - acc: 0.9294 - val_loss: 0.2215 - val_acc: 0.9247
Epoch 7/10
200/200 [==============================] - 275s 1s/step - loss: 0.2110 - acc: 0.9356 - val_loss: 0.2120 - val_acc: 0.9390
Epoch 8/10
200/200 [==============================] - 276s 1s/step - 

This method yielded a slightly lower validation accuracy than the first model. This does show that we don't quite need as many layers as at first sight. We also don't have any Dropout layers which is also very interesting. My guess is that since our domain of possible images is so small, the network would overfit to any other type of image, but not the one's we're feeding it.

### Network without a CNN

This next network will be a neural network with no convolutions. This will most definitely yield poorer results, just based on the current literature, but should be interesting nontheless.

In [40]:
model = Sequential()
model.add(Reshape((110, 110, 3), input_shape=(110, 110, 3)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

In [41]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 110, 110, 3)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 36300)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 1024)              37172224  
_________________________________________________________________
activation_8 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_9 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 1026      
__________

In [42]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 126s 629ms/step - loss: 8.1834 - acc: 0.4906 - val_loss: 8.1299 - val_acc: 0.4956
Epoch 2/10
200/200 [==============================] - 128s 641ms/step - loss: 8.0062 - acc: 0.5033 - val_loss: 8.0034 - val_acc: 0.5035
Epoch 3/10
200/200 [==============================] - 133s 663ms/step - loss: 7.8047 - acc: 0.5158 - val_loss: 8.0944 - val_acc: 0.4978
Epoch 4/10
200/200 [==============================] - 130s 648ms/step - loss: 8.1976 - acc: 0.4914 - val_loss: 7.9958 - val_acc: 0.5039
Epoch 5/10
200/200 [==============================] - 131s 653ms/step - loss: 7.9994 - acc: 0.5037 - val_loss: 8.0767 - val_acc: 0.4989
Epoch 6/10
200/200 [==============================] - 138s 692ms/step - loss: 8.0742 - acc: 0.4991 - val_loss: 8.0742 - val_acc: 0.4991
Epoch 7/10
200/200 [==============================] - 122s 609ms/step - loss: 8.0716 - acc: 0.4992 - val_loss: 7.9956 - val_acc: 0.5039
Epoch 8/10
200/200 [============================

As expected, the accuracy of this model is attrocious. It's basically as if we flipped a coin. Let's try some additions to this non-CNN image classifier network

### Pooling without Convolution

Just as a shot in the dark, let's add some MaxPooling to our layers, but still not use a Convolutional Layer at all

In [44]:
model = Sequential()
model.add(Reshape((110, 110, 3), input_shape=(110, 110, 3)))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

In [45]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 110, 110, 3)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 3)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3888)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 1024)              3982336   
_________________________________________________________________
activation_11 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
__________

In [48]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 80s 398ms/step - loss: 8.2253 - acc: 0.4897 - val_loss: 8.1299 - val_acc: 0.4956
Epoch 2/10
200/200 [==============================] - 69s 345ms/step - loss: 7.9986 - acc: 0.5038 - val_loss: 8.0034 - val_acc: 0.5035
Epoch 3/10
200/200 [==============================] - 68s 342ms/step - loss: 7.8198 - acc: 0.5148 - val_loss: 8.0944 - val_acc: 0.4978
Epoch 4/10
200/200 [==============================] - 70s 348ms/step - loss: 8.1925 - acc: 0.4917 - val_loss: 7.9958 - val_acc: 0.5039
Epoch 5/10
200/200 [==============================] - 55s 275ms/step - loss: 7.9666 - acc: 0.5057 - val_loss: 8.0767 - val_acc: 0.4989
Epoch 6/10
200/200 [==============================] - 56s 278ms/step - loss: 8.0868 - acc: 0.4983 - val_loss: 8.0742 - val_acc: 0.4991
Epoch 7/10
200/200 [==============================] - 53s 267ms/step - loss: 8.0943 - acc: 0.4978 - val_loss: 7.9956 - val_acc: 0.5039
Epoch 8/10
200/200 [==============================] - 5

## Summary

Seems like the best performing model was the first one we had, with a validation accuracy score of 0.94. The only other good model was a derivation of the first. The transfer learning was poor (not generalized enough to our domain) and I had **no hope** for the non-convolution networks. What is interesting is that even when we remove most of our first network (Pooling, Dropout, multiple convolutions) we still get basically the same validation accuracy score of around 0.93. My guess is that the first convolution find the edges of the malaria, no further steps required. We could output the result of the convolution, but I don't want to fiddle around with that. Interesting nonetheless.